# 8. 아이유팬이 좋아할 만한 다른 아티스트 찾기

## 1. 들어가며

오늘은 머신러닝 어플리케이션 중 가장 상업적인 성공을 거둔 것으로 평가받고 있는 __추천시스템(Recommender System)__에 대해 알아보려고 한다. 어느덧 추천시스템은 우리 생활 깊숙이 들어와 있다. 유튜브, 넷플릭스 등 동영상 플랫폼이 기가막히게 내가 좋아할 만한 콘텐츠를 찾아서 자동으로 플레이해 주고 있다. 아마존, 쿠팡 온라인 쇼핑 사이트에서도 내 취향에 맞는 상품추천을 쉽게 접할 수 있다. 페이스북, 인스타그램 같은 SNS는 말할 것도 없고, 어쩌면 느끼지도 못하셨겠지만 뉴스나 광고까지도 추천시스템의 원리가 숨어들어 있다.<br><br>

이런 추천시스템의 원리를 한 문장으로 줄인다면 __나와 비슷한 다른 사용자들이 좋아하는 것과 비슷한 것을 내게 추천해 준다__ 일 것이다. 하지만 어떤 사용자들이 나와 비슷한지를 어떻게 알 수 있을까? 또 어떤 상품이나 정보가 서로 유사한지는 어떻게 알 수 있는 걸까? 오늘의 과정을 통해 추천시스템의 기본 원리를 파악해 보고 나만의 추천시스템을 만들 수 있는 멋진 아이디어를 떠올려 보자.

### 학습 목표
---
> 0. 추천시스템의 개념과 목적을 이해한다.
> 1. Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
> 2. 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
> 3. 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
> 4. 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
> 5. 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.

### 목차
---
1. 들어가며
2. 데이터 탐색하기와 전처리
3. 사용자의 명시적/암묵적 평가
4. Matrix Factorization(MF)
5. CSR(Compressed Sparse Row) Matrix
6. MF 모델 학습하기
7. 비슷한 아티스트 찾기 + 유저에게 추천하기

## 2. 추천시스템이란 게 뭔가요?
2020년 현재 온라인 콘텐츠 서비스에서 데이터분석과 AI기술을 접목한 추천시스템의 활용은 이제 선택이 아닌 필수가 된 것 같다. 우리가 흔히 사용하고 있는 음원 서비스를 예로 들어보자. 아래 글을 읽어보면 이미 추천시스템을 통한 개인화가 서비스 전면으로 나서는 트렌드의 변화가 이미 우리 곁에 다가와 있다는 것을 알 수 있을 것이다.

[치열한 음원 큐레이션 서비스 경쟁](https://blog.naver.com/businessinsight/222191549425)

유튜브, 애플뮤직, 멜론 등 음원 서비스 제공자들은 수많은 유저들이 어떤 아티스트의 노래를 들었는지에 대한 광범위한 데이터를 축적하고 있다. 이러한 빅데이터가 정확한 추천의 원동력이 되고 있다는 것은 어렴풋이 짐작할 수 있을 것이다. 그 추천의 원리가 어떤 것인지 좀 더 구체적으로 파악하고 싶다면 아래 링크의 글을 읽어보자.

[콘텐츠 추천 알고리즘의 진화](http://www.kocca.kr/insight/vol05/vol05_04.pdf)

__협업 필터링(Collaborative Filtering) 방식과 콘텐츠 기반 필터링(Contents-based Filtering) 방식의 차이점__

* 협업 필터링은 다수의 사용자가 아이템을 구매한 이력 정보만으로 사용자간 유사성 및 아이템 간 유사성을 파악하지만, 콘텐츠 기반 필터링은 아이템의 고유의 정보를 바탕으로 아이템 간 유사성을 파악한다. 다시 말해, 협업 필터링에서는 아이템과 사용자 간의 행동 또는 관계에만 주목할 뿐 아이템 자체의 고유한 속성에 주목하지 않으나, 콘텐츠 기반 필터링에서는 아이템 자체의 속성에 주목한다. 하지만 반대로, 콘텐츠 기반 필터링은 사용자와 아이템 간의 관련성에 주목하지 않으므로, 사용자의 특성에 따른 개인화된 추천을 제공하기 어렵다.

__다수의 사용자의 판단을 기반으로 정확한 추천을 가능하게 하는 추천시스템의 핵심 근간은 협업 필터링이지만, 협업 필터링을 바로 사용할 수 없게 만드는 세가지 제약 조건이 있다.__

* 1)시스템이 충분한 정보를 모으지 못한 사용자나 아이템에 대한 추론을 할 수 없는 상태인 콜드 스타트(Cold Start) 상황 
* 2)계산량이 너무 많아 추천의 효율이 떨어지는 상황 
* 3)롱테일의 꼬리 부분, 즉 사용자의 관심이 저조한 항목의 정보가 부족하여 추천에서 배제되는 상황

__유튜브 뮤직의 첫화면에서 처음 접속한 사용자에게 좋아하는 아티스트 5명 이상의 정보를 요구하는 이유는 무엇일까?__

* 처음 접속한 사용자라면 이 사용자에 대한 아무런 초기 정보가 없는 상황이어서 사용자 기반 추천을 전혀 할 수 없어 콘텐츠 기반 필터링 방식의 추천만 가능한데, 이를 보완하여 처음부터 사용자 특성에 따른 맞춤형 서비스를 제공하기 위해서이다. 사용자 맞춤형 추천을 제공하지 못한다면 모든 초기 사용자에게 같은 화면을 제시할 수밖에 없는데, 이러면 모든 사용자들이 처음 추천받은 콘텐츠 위주로만 시청하게 되어 추천시스템이 빠지기 쉬운 필터 버블의 문제을 야기하거나, 추천의 다양성을 저해할 수 있는 위험이 생긴다.

추천시스템은 이처럼 <span style="color:red">a)아이템은 매우 많고 </span><span style="color:blue">b)유저의 취향은 다양할 때</span> 유저가 소비할만한 아이템을 예측하는 모델이다.<br>
몇 가지 실제 사례를 보면,
* 유튜브 :<span style="color:red"> 동영상이 매일 엄청나게 많이 올라오고 </span><span style="color:blue">유저의 취향(게임 선호, 뷰티 선호, 지식 선호, 뉴스 선호)이 다양하다.</span>
* 페이스북 :<span style="color:red">포스팅되는 글이 엄청 많고 </span><span style="color:blue">유저가 관심 있는 페이지, 친구, 그룹은 전부 다르다.</span>
* 아마존 : <span style="color:red">카테고리를 한정해도 판매 품목이 엄청 많고 </span><span style="color:blue">좋아하는 브랜드, 구매 기준이 다양하다.</span>

아티스트는 수없이 많고 새로운 아티스트가 계속 나오기 때문에, 유튜브 같은 추천 기반의 미디어 회사들은 꾸준히 아이템이 되는 아티스트 정보를 자동으로 수집하고, 사용자들이 아티스트를 선택한 정보를 업데이트하여 축적하고 있다. 이렇게 쌓인 방대한 데이터가 정확한 협업 필터링을 위한 기반이 된다.

이렇게 쌓인 데이터 사례 중 하나를 활용하여 유저가 좋아하는 특정 아티스트와 유사한 다른 아티스트를 추천하는 추천시스템을 만들어 보려고 한다. [Last.fm](https://www.last.fm/)에서는 어떤 유저가 특정 아티스트의 노래를 몇 번이나 들었는지에 대한 [데이터](http://ocelma.net/MusicRecommendationDataset/lastfm-360K.html)를 제공하고 있다. 2010년에 미국에서 서비스되고 있는 spotify에서 발생한 데이터이기 때문에 생소한 아티스트가 많다.

그럼 아래와 같이 작업 디렉토리를 생성하고, 데이터셋을 다운로드받아 실습환경을 구성해 보자. 참고로, 오늘 사용하게 될 데이터셋은 압축을 풀기 전에 543MB, 압축을 풀고 나면 1.6GB가 넘어가는 꽤 큰 용량을 자랑한다. 인터넷 환경에 따라 다운로드 시간이 4시간 이상 소요될 수 있다.

```bash
1) 작업디렉토리 생성
$ mkdir -p ~/aiffel/recommendata_iu/data

2) wget으로 데이터 다운로드 (주의) 오래걸립니다. 
$ wget http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz

3) 다운받은 데이터를 작업디렉토리로 옮기고, 작업디렉토리로 이동합니다.
$ mv lastfm-dataset-360K.tar.gz ~/aiffel/recommendata_iu/data & cd ~/aiffel/recommendata_iu/data

4) gzip으로 압축된 압축을 해제하면 tar 파일이 하나 나옵니다. 
$ gunzip lastfm-dataset-360K.tar.gz

5) tar 압축을 다시 해제하면 우리가 사용할 최종 데이터 파일이 나옵니다. 
$ tar -xvf lastfm-dataset-360K.tar

6) 필요 라이브러리 설치
$ pip install implicit
```

(참고) 위 4)~5) 과정은 아래와 같이 z 옵션을 줘서 한꺼번에 처리 가능하다. tar 사용법에 대해서는 [여기](https://nota.tistory.com/53)를 참고

```
$ tar -xvzf lastfm-dataset-360K.tar.gz
```

오늘 사용하게 될 미국 데이터로는 '아이유팬이 좋아할 만한 다른 아티스트'를 추천할 수는 없다. 하지만 최근에 카카오에서 학술용으로만 공개한 [멜론 데이터](https://arena.kakao.com/c/7)를 활용한다면 오늘 학습한 방법으로 정말 아이유와 관련된 추천을 할 수 있을 것이다.

## 3. 데이터 탐색하기와 전처리

### 데이터 준비
---
데이터를 열어서 몇가지 확인 및 전처리를 진행해 보자.

다룰 데이터는 그동안 흔히 보던 `csv` 파일이 아니고 `tsv` 파일이다. 어떻게 생긴 파일일까? 다음과 같이 열어 보자.

```bash
$ more ~/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv
```

아마 아래 그림과 같이 보일 것이다. `tsv`는 Tab-Separated Values의 약자로서, Comma-Separated Values의 약자인 `csv` 파일과 거의 동일하나 구분자만 comma가 아니라 tab('\t') 문자를 활용한 것이다. 이런 텍스트 파일은 Pandas의 read_csv 메소드로도 충분히 파싱해서 열어볼 수 있다.

<img src="./image/tsv.png" alt="tsv" width="70%"/>

__위 그림을 보면 행마다 탭으로 구분된 4개의 컬럼이 포함된 것 같다. 데이터셋 홈페이지(http://ocelma.net/MusicRecommendationDataset/lastfm-360K.html)에서 확인해 볼 때, 4개의 컬럼의 의미는 각각 무엇일까?__

* 항목정의가 ( user-mboxsha1 \t musicbrainz-artist-id \t artist-name \t plays ) 라고 되어 있는 것으로 보아, 4개의 항목은 각각 User ID, Artist MBID, Artist Name, Play횟수 로 보인다.

그럼 Pandas를 활용해 이 데이터를 열어 보겠다. 위 데이터에는 컬럼명이 포함되어 있지 않으므로, `col_names`에 적절히 컬럼명을 지정해 주도록 하자.

In [1]:
import pandas as pd
import os

fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

user_id  \
0  00000c289a1829a808ac09c00daf10bc3c4e223b   
1  00000c289a1829a808ac09c00daf10bc3c4e223b   
2  00000c289a1829a808ac09c00daf10bc3c4e223b   
3  00000c289a1829a808ac09c00daf10bc3c4e223b   
4  00000c289a1829a808ac09c00daf10bc3c4e223b   
5  00000c289a1829a808ac09c00daf10bc3c4e223b   
6  00000c289a1829a808ac09c00daf10bc3c4e223b   
7  00000c289a1829a808ac09c00daf10bc3c4e223b   
8  00000c289a1829a808ac09c00daf10bc3c4e223b   
9  00000c289a1829a808ac09c00daf10bc3c4e223b   

                            artist_MBID                   artist  play  
0  3bd73256-3905-4f3a-97e2-8b341527f805          betty blowtorch  2137  
1  f2fb0ff0-5679-42ec-a55c-15109ce6e320                die Ärzte  1099  
2  b3ae82c2-e60b-4551-a76d-6620f1b456aa        melissa etheridge   897  
3  3d6bbeb7-f90e-4d10-b440-e153c0d10b53                elvenking   717  
4  bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8     juliette & the licks   706  
5  8bfac288-ccc5-448d-9573-c33ea2aa5c30    red hot chili peppers   691  
6  6531c8b1-76ea-4141-b270-eb1ac5b41375                   magica   545  
7  21f3573f-10cf-44b3-aeaa-26cccd8448b5  the black dahlia murder   507  
8  c5db90c4-580d-4f33-b364-fbaa5a3a58b5              the murmurs   424  
9  0639533a-0402-40ba-b6e0-18b067198b73               lunachicks   403

`artist_MBID` 항목은 우리의 관심사가 아니기 때문에, 사용하는 컬럼만 남기고 제거하도록 하자.

In [2]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

user_id                   artist  play
0  00000c289a1829a808ac09c00daf10bc3c4e223b          betty blowtorch  2137
1  00000c289a1829a808ac09c00daf10bc3c4e223b                die Ärzte  1099
2  00000c289a1829a808ac09c00daf10bc3c4e223b        melissa etheridge   897
3  00000c289a1829a808ac09c00daf10bc3c4e223b                elvenking   717
4  00000c289a1829a808ac09c00daf10bc3c4e223b     juliette & the licks   706
5  00000c289a1829a808ac09c00daf10bc3c4e223b    red hot chili peppers   691
6  00000c289a1829a808ac09c00daf10bc3c4e223b                   magica   545
7  00000c289a1829a808ac09c00daf10bc3c4e223b  the black dahlia murder   507
8  00000c289a1829a808ac09c00daf10bc3c4e223b              the murmurs   424
9  00000c289a1829a808ac09c00daf10bc3c4e223b               lunachicks   403

검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꾸자.

In [3]:
data['artist'] = data['artist'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

user_id                   artist  play
0  00000c289a1829a808ac09c00daf10bc3c4e223b          betty blowtorch  2137
1  00000c289a1829a808ac09c00daf10bc3c4e223b                die ärzte  1099
2  00000c289a1829a808ac09c00daf10bc3c4e223b        melissa etheridge   897
3  00000c289a1829a808ac09c00daf10bc3c4e223b                elvenking   717
4  00000c289a1829a808ac09c00daf10bc3c4e223b     juliette & the licks   706
5  00000c289a1829a808ac09c00daf10bc3c4e223b    red hot chili peppers   691
6  00000c289a1829a808ac09c00daf10bc3c4e223b                   magica   545
7  00000c289a1829a808ac09c00daf10bc3c4e223b  the black dahlia murder   507
8  00000c289a1829a808ac09c00daf10bc3c4e223b              the murmurs   424
9  00000c289a1829a808ac09c00daf10bc3c4e223b               lunachicks   403

첫 번째 유저가 어떤 아티스트의 노래를 듣는지 확인해보자.

In [4]:
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

user_id                   artist  play
0   00000c289a1829a808ac09c00daf10bc3c4e223b          betty blowtorch  2137
1   00000c289a1829a808ac09c00daf10bc3c4e223b                die ärzte  1099
2   00000c289a1829a808ac09c00daf10bc3c4e223b        melissa etheridge   897
3   00000c289a1829a808ac09c00daf10bc3c4e223b                elvenking   717
4   00000c289a1829a808ac09c00daf10bc3c4e223b     juliette & the licks   706
5   00000c289a1829a808ac09c00daf10bc3c4e223b    red hot chili peppers   691
6   00000c289a1829a808ac09c00daf10bc3c4e223b                   magica   545
7   00000c289a1829a808ac09c00daf10bc3c4e223b  the black dahlia murder   507
8   00000c289a1829a808ac09c00daf10bc3c4e223b              the murmurs   424
9   00000c289a1829a808ac09c00daf10bc3c4e223b               lunachicks   403
10  00000c289a1829a808ac09c00daf10bc3c4e223b         walls of jericho   393
11  00000c289a1829a808ac09c00daf10bc3c4e223b           letzte instanz   387
12  00000c289a1829a808ac09c00daf10bc3c4e223b                goldfrapp   361
13  00000c289a1829a808ac09c00daf10bc3c4e223b               horrorpops   358
14  00000c289a1829a808ac09c00daf10bc3c4e223b             the butchies   329
15  00000c289a1829a808ac09c00daf10bc3c4e223b            jack off jill   316
16  00000c289a1829a808ac09c00daf10bc3c4e223b         babes in toyland   310
17  00000c289a1829a808ac09c00daf10bc3c4e223b         dropkick murphys   302
18  00000c289a1829a808ac09c00daf10bc3c4e223b            all:my:faults   288
19  00000c289a1829a808ac09c00daf10bc3c4e223b                 le tigre   281
20  00000c289a1829a808ac09c00daf10bc3c4e223b               schandmaul   244
21  00000c289a1829a808ac09c00daf10bc3c4e223b                    edguy   232
22  00000c289a1829a808ac09c00daf10bc3c4e223b      maximum the hormone   231
23  00000c289a1829a808ac09c00daf10bc3c4e223b                 all ends   229
24  00000c289a1829a808ac09c00daf10bc3c4e223b             jack johnson   227
25  00000c289a1829a808ac09c00daf10bc3c4e223b                eluveitie   222
26  00000c289a1829a808ac09c00daf10bc3c4e223b                rasputina   220
27  00000c289a1829a808ac09c00daf10bc3c4e223b    london after midnight   210
28  00000c289a1829a808ac09c00daf10bc3c4e223b              the killers   208
29  00000c289a1829a808ac09c00daf10bc3c4e223b                  mutyumu   205
30  00000c289a1829a808ac09c00daf10bc3c4e223b             judas priest   198
31  00000c289a1829a808ac09c00daf10bc3c4e223b               rob zombie   198
32  00000c289a1829a808ac09c00daf10bc3c4e223b             the bosshoss   189
33  00000c289a1829a808ac09c00daf10bc3c4e223b         blue öyster cult   185
34  00000c289a1829a808ac09c00daf10bc3c4e223b             sandra nasic   183
35  00000c289a1829a808ac09c00daf10bc3c4e223b               john mayer   182
36  00000c289a1829a808ac09c00daf10bc3c4e223b           sleater-kinney   175
37  00000c289a1829a808ac09c00daf10bc3c4e223b                  the who   168
38  00000c289a1829a808ac09c00daf10bc3c4e223b                 disciple   154
39  00000c289a1829a808ac09c00daf10bc3c4e223b                  tanzwut   153
40  00000c289a1829a808ac09c00daf10bc3c4e223b               guano apes   151
41  00000c289a1829a808ac09c00daf10bc3c4e223b       the rolling stones   150
42  00000c289a1829a808ac09c00daf10bc3c4e223b          little big town   145
43  00000c289a1829a808ac09c00daf10bc3c4e223b              team dresch   137
44  00000c289a1829a808ac09c00daf10bc3c4e223b                    betty   135
45  00000c289a1829a808ac09c00daf10bc3c4e223b                       l7   135
46  00000c289a1829a808ac09c00daf10bc3c4e223b                bif naked   134
47  00000c289a1829a808ac09c00daf10bc3c4e223b               girlschool   134
48  00000c289a1829a808ac09c00daf10bc3c4e223b          the wallflowers   131

처음 들어보는 아티스트가 많다. 추천시스템에서는 적용하는 분야에 대한 지식이 특히 더 필요한데, 모르면 검증을 못 하기 때문이다.

### 데이터 탐색
---
추천 모델을 만들기 전에 데이터의 기본적인 정보를 보자.

* 유저수, 아티스트수, 인기 많은 아티스트
* 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
* 유저 play 횟수 중앙값에 대한 통계

(참고) `pandas.DataFrame.nunique()`은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용하다.

In [5]:
# 유저 수
data['user_id'].nunique()

358868

In [6]:
# 아티스트 수
data['artist'].nunique()

291346

In [7]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [8]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [9]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

### 모델 검증을 위한 사용자 초기 정보 세팅
---
유튜브 뮤직 등 추천시스템들은 이를 위해서 처음 가입하는 사용자의 취향과 유사한 아티스트 정보를 5개 이상 입력받는 과정을 거치게 하는 경우가 많다.<br>
지금도 동일한 과정이 필요하다. 만든 추천시스템의 추후 검증 과정을 위해, '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주어야 한다. 아래와 같이 내가 좋아하는 유명한 아티스트들을 기존의 데이터에 추가해서 넣어보겠다.



In [10]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

user_id           artist  play
17535650  sep 20, 2008       turbostaat    12
17535651  sep 20, 2008    cuba missouri    11
17535652  sep 20, 2008  little man tate    11
17535653  sep 20, 2008        sigur rós    10
17535654  sep 20, 2008       the smiths    10
0                zimin  black eyed peas    30
1                zimin          maroon5    30
2                zimin       jason mraz    30
3                zimin         coldplay    30
4                zimin          beyoncé    30

### 모델에 활용하기 위한 전처리 (실습)
---
사람이 태어나면 주민등록번호가, 학교에 가면 출석번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여준다. 현재 다루는 데이터에서는 user와 artist 각각에 번호를 붙이려고 한다. 보통 이런 작업을 __indexing__이라고 한다. 추천시스템, 자연어처리에서 자주 하는 작업들이다.

(참고) `pandas.DataFrame.unique()`은 특정 컬럼에 포함된 유니크한 데이터만 모아 준다. indexing 작업을 위해 매우 유용하다.



In [11]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [12]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [13]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


user_id  artist  play
0         0       0  2137
1         0       1  1099
2         0       2   897
3         0       3   717
4         0       4   706
..      ...     ...   ...
0    358868     376    30
1    358868  270115    30
2    358868    3746    30
3    358868      62    30
4    358868     396    30

[17535660 rows x 3 columns]

`data`의 `user_id`와 `artist` 컬럼 내 값들이 모두 정수 인덱스 값으로 잘 변경되었다면, 훈련을 위한 전처리가 완료되었다.

## 4. 사용자의 명시적/암묵적 평가
만들고 있는 추천시스템은 사용자들이 아이템을 얼마나 선호하는지를 모델링하기를 원한다. 그러자면 사용자의 아이템 선호도를 말해 주는 유저 행동 데이터셋이 필요하다. 만약 우리 데이터가 __좋아요__나 __별점__처럼 선호도를 명시적(explicit)으로 나타내는 것이라면 참 좋겠지만, 지금 우리가 가지고 있는 데이터는 사용자가 아티스트의 곡을 몇 번 플레이했나 하는 것뿐이다. 이렇게 서비스를 사용하면서 자연스럽게 발생하는 암묵적(implicit)인 피드백도 사용자의 아이템에 대한 평가를 알 수 있는 단서가 될 수 있다.

(참조)[Explicit vs. Implicit Feedback Datasets](https://orill.tistory.com/entry/Explicit-vs-Implicit-Feedback-Datasets?category=1066301)

__암묵적 평가가 될 수 있는 데이터의 사례__

* 예시로 든 플레이 횟수, 플레이 시간 이외에도 클릭 수, 구매 여부, 플레이 스킵 여부, 검색 기록, 방문 페이지 이력, 구매 내역, 심지어 마우스 움직임 기록도 암묵적 평가 데이터셋이 될 수 있다.

__어떤 유저가 아티스트의 곡을 한 번만 들었다는 것의 의미를 생각해보자. 유저는 이 아티스트를 좋아하는 걸까 싫어하는 걸까? 이 유저에게 이 아티스트와 관련된 사람을 추천해야 할까?__

* 이것은 명확한 정답은 존재하지 않는다. 어떤 유저는 한 번 듣고 별로라고 생각했을 수도 있고, 어떤 유저는 다시 듣고 싶지만 가수의 이름을 잊어버렸을 수도 있다. 이런 애매한 암묵적 데이터야말로 도메인 지식과 직관이 활용되어야 하는 영역이다.

그렇다면 데이터셋에서 1회만 플레이한 데이터의 비율은 어떻게 될까?

In [14]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


앞으로 만들어갈 모델에서는 암묵적 데이터의 해석을 위해 다음과 같은 규칙을 적용할 것이다.

> 1. 한 번이라도 들었으면 선호한다고 판단한다.
> 2. 많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.

## 5. Matrix Factorization(MF)
데이터가 준비되었으니 이제 모델을 만들 차례!

아래 그림의 왼쪽에서 보는 것처럼, m명의 사용자들이 n명의 아티스트에 대해 평가한 데이터를 포함한 (m,n) 사이즈의 평가행렬(Rating Matrix)을 만들 것이다. 행렬 중 일부는 데이터가 채워져 있지만, 나머지 부분은 데이터가 비어 있다. 앞에서 살펴보았던 추천시스템의 `협업 필터링(Collaborative Filtering)`이란 결국 이런 평가행렬을 전제로 하는 것이다. 만약 이 평가행렬의 비어 있는 부분을 포함한 완벽한 정보를 얻을 수만 있다면 (모든 사용자의 모든 아이템에 대한 선호도를 다 파악한 셈이니) 그야말로 완벽한 추천이 가능해질 것이다.

추천 시스템의 다양한 모델 중 `Matrix Factorization(MF, 행렬분해)` 모델을 사용하겠다. `MF` 모델은 2006년 Netflix에서 백만달러의 상금을 걸고 개최한 자사 추천시스템의 성능을 10% 이상 향상시키는 챌린지를 계기로 알려지게 되었다. 기본 아이디어는 아래 그림과 같다. (m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다는 간단한 아이디어이다. 대체로 k는 m이나 n보다 훨씬 작은 값이기 때문에 계산량 측면으로도 훨씬 유리해진다. 아이디어의 단순함에도 불구하고 MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용되는 모델이다.

<img src="./image/mf.png">

아래 그림을 다시 한번 보자. 위에 소개한 MF 모델의 개요도와 크게 다를 것은 없다. 단지, MF 모델을 사용자에게 영화를 추천하는 모델에 대입해서 그려 보았다는 정도의 차이가 있다. 위 그림 방식으로 말하자면 m=4, n=5, k=2인 MF 모델인 것이다.

위에서 MF 모델은 큰 평가행렬 R을 두 개의 Feature Matrix P와 Q로 분해한다고 하였다.<br>
무엇의 Feature일까? (m, k) 사이즈의 Feature Matrix P는 k 차원의 벡터를 사용자 수만큼 모아놓은 행렬이다. 그렇다면 아래 그림에서 첫 번째 벡터 $P_{0}$=(1, 0.1)은 바로 빨간 모자를 쓴 첫 번째 __사용자의 특성(Feature) 벡터__가 된다.<br>
같은 방식으로 Q 행렬의 첫번째 벡터 $Q_{0}$=(0.9, -0.2)는 해리포터 __영화의 특성 벡터__가 되는 것이다. MF 모델은 이 두 벡터를 내적해서 얻어지는 0.88이 바로 $R_{0,0}$ 으로 정의되는 사용자의 영화 선호도로 보는 모델이다.

<img src="./image/mf2.png">

모델의 목표는 모든 유저와 아이템에 대해 k-dimension의 벡터를 잘 만드는 것이다.<br>
벡터를 잘 만드는 기준은 __유저i의 벡터($U_{i}$)와 아이템j의 벡터($I_{j}$)를 내적했을 때 유저i가 아이템j에 대해 평가한 수치($M_{ij}$)와 비슷한지__ 이다.

$$U_{i}I_{j} = M_{ij}$$

MF에도 다양한 변형이 있다. 이번에 사용할 모델은 [Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf) 논문에서 제안한 모델을 사용할 것이다.

이 모델의 경우 'black eyed peas'를 play했다는 데이터를 추가했기 때문에 그 벡터와 black eyed peas의 벡터를 곱했을 때 1에 가까워야 모델이 잘 학습하는 것을 목표로 한다.

오해할 수 있는데, black eyed peas를 5번 들었어도 모델이 근사하고자 하는 것이 <span style="color:red">아티스트를 선호하는지 안 하는지를 맞추는 것</span>이기 때문에 두 벡터를 곱했을 때 5에 가까워지는 대신 1에 가까워져야 한다. 이를 통해서 내가 들어보지 못한 queen에 대해서도 <span style="color:red">나의 벡터와 queen의 벡터를 곱하여 수치를 예상할 수 있다.</span>

만약 목표가 유저의 재생 횟수를 맞춰야 하는 것(혹은 별점을 맞추는 것)이라면 그에 맞는 다른 모델을 사용해야 한다.

구조가 거의 똑같고 모델의 목표가 재생횟수를 맞춰지는 쪽으로 학습(유저와 아이템의 벡터를 만들기)하면 된다! 궁금하다면 [참고자료](https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b)를 확인.

## 6. CSR(Compressed Sparse Row) Matrix
유저 X 아이템 평가행렬을 다시 생각해 보자. 유저는 36만 명이고 아티스트는 29만 명이다. 이를 행렬로 표현하고 행렬의 각 원소에 정수 한 개 (1byte)가 들어간다면 36만 * 29만 * 1byte $\approx$ <span style="color:red">97GB</span>가 필요하다.

__평가행렬 용량이 이렇게 커진 이유는 무엇일까?__

* 유저 수 X 아티스트 수만큼의 정보 안에는 유저가 들어보지 않은 아티스트에 대한 정보까지 모두 행렬에 포함되어 계산되기 때문이다. 아무리 많은 아티스트를 들어 본 유저도 수백 명을 넘어갈 만큼의 아티스트를 들어보지 못했는데, 총 아티스트는 29만 명이 넘기 때문에 평가행렬 내의 대부분의 공간은 0으로 채워진다. 이런 행렬을 Sparse Matrix라고 한다. 이런 메모리 낭비를 최소화하기 위해서는 유저가 들어본 아티스트에 대해서만 정보만을 저장하면서 전체 행렬 형태를 유추할 수 있는 데이터 구조가 필요하다.

참고로 대부분이 사용하는 컴퓨터의 메모리는 많아야 16GB일 테니 97GB나 되는 거대한 행렬을 메모리에 올려놓고 작업한다는 것은 불가능할 것이다. 이런 경우의 좋은 대안이 되는 것이 `CSR(Compressed Sparse Row) Matrix`이다. 다음 스텝에서 수행할 모델 학습의 input으로 사용할 데이터 타입을 CSR Matrix로 할 것이다.<br>
CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조이다. CSR에 대한 자세한 설명은 다음 링크를 참고. 혹시 설명이 부족하면 그 아래 StackOverflow의 설명이 보완적인 역할을 해줄 것이다.

[Scipy sparse matrix handling](https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/#csr-matrix)

[StackOverflow csr_matrix 설명](https://stackoverflow.com/questions/53254104/cant-understand-scipy-sparse-csr-matrix-example/62118005#62118005)

* 이제 data를 CSR Matrix에 맞게 바꿔보자.<br>
* DataFrame을 만드는 방식이 다양하듯 csr_matrix를 [만드는 방법](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)은 다양하게 있다.<br>
* 현재 data와 같은 데이터 구조에 적합한 방식은 위 링크에서 4번째로 설명하고 있는 방법이다.

<img src="./image/scipy.png" />
<center>https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html</center>

```python
csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])  
where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]., M,N은 matrix의 shape
```

In [15]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

## 7. MF 모델 학습하기
이전 스텝에서 설명한 Matrix Factorization 모델을 [implicit](https://github.com/benfred/implicit) 패키지를 사용하여 학습해보자.

* `implicit` 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지이다.
* 이 패키지에 구현된 `als(AlternatingLeastSquares) 모델`을 사용하겠다. `Matrix Factorization`에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있다.


In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 \__init__ 파라미터를 살펴보자. 
1. __factors__ : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. __regularization__ : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. __use_gpu__ : GPU를 사용할 것인지 
4. __iterations__ : epochs와 같은 의미. 데이터를 몇 번 반복해서 학습할 것인지

1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 한다.

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

* 모델 학습이 끝났다!
* 모델이 1)나의 벡터와 black eyed peas의 __벡터를 어떻게 만들고 있는지__와 2)두 벡터를 __곱하면 어떤 값__이 나오는지 살펴보자.

In [20]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

In [21]:
zimin_vector

array([-1.3217945 ,  1.1512406 ,  0.01459718, -0.22784264,  1.0407428 ,
        0.6512222 , -0.24383156,  0.8360913 ,  0.1030971 ,  0.25013444,
       -0.39967865, -0.78811324,  0.5224154 ,  0.13651064,  1.3847893 ,
       -0.00399836, -0.2859625 , -0.612104  ,  0.7894685 ,  0.67711335,
        0.43401414,  0.18184695, -0.03368484,  1.1228015 ,  0.01773584,
       -0.977355  ,  0.71690935,  1.0793767 ,  0.13008906,  0.31587887,
        0.18192406, -0.73810786, -0.15127963, -0.14942193,  0.66913706,
        0.3677542 ,  1.2576709 ,  0.27324295, -0.8430803 , -0.28335536,
        0.29941815, -0.4586004 ,  1.1104715 , -0.89961934, -0.650566  ,
        0.22611307,  0.5971996 ,  0.01525323, -0.23713966, -0.41181776,
       -0.41353047,  0.8839887 , -0.56926745, -0.39764664, -0.27962482,
        0.99266624, -0.0716833 , -0.39318854, -0.5573965 ,  0.37409866,
        0.17414102, -0.3749985 ,  1.3845518 ,  0.6018494 ,  0.78888613,
        0.22065364,  0.26217043,  0.5301825 , -0.41458628, -0.46

In [22]:
black_eyed_peas_vector

array([-0.00440142,  0.01425587,  0.01032706,  0.00794074,  0.02009073,
        0.01126088,  0.01616257,  0.02157471,  0.01586687,  0.01146659,
        0.00988837, -0.00457437,  0.0198268 , -0.00052022,  0.02437137,
        0.01140233, -0.01376139,  0.00838831, -0.00344924,  0.01123847,
        0.02230747,  0.01312981,  0.00954666,  0.02969722,  0.006698  ,
        0.00033924,  0.00863514,  0.03065595,  0.00552109,  0.01418612,
        0.00571874, -0.00866555,  0.01593849,  0.00283078,  0.00938751,
       -0.00261478,  0.04646141, -0.00098492,  0.00338087, -0.00438647,
        0.0091673 ,  0.01265567,  0.01735165,  0.01420306, -0.0099524 ,
        0.01431996,  0.01566991, -0.00077696,  0.00337259,  0.00213553,
        0.00739379,  0.00739587,  0.00773828,  0.00396929, -0.00642766,
        0.00819549,  0.00903346, -0.01553026,  0.00533701,  0.00146315,
        0.01993659,  0.00124256,  0.03237496,  0.02574438,  0.01042388,
        0.0124958 ,  0.02501645,  0.01423079,  0.00293744,  0.00

In [23]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.52491635

1이 나와야 될 것 같은데 한참 낮은 수치인 0.49정도가 나왔다. factors를 늘리거나 iterations를 늘려야 할 것 같다.

__만약 이 두 수치를 늘려 1에 가깝게 나왔다면 이 모델이 잘 학습되었다고 볼 수 있을까?__

* 정답은 아직 검증되지 않았다이다. 학습 데이터에 대해서 fitting은 되었지만 아직 보지 못한 데이터에 대한, 예를 들어 queen을 선호할지에 대한 예측을 잘할지 못할지는 검증하지 않았기 때문이다.

모델이 queen에 대한 선호도를 어떻게 예측할지 보자.

In [24]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.32332128

__queen에 대한 선호도를 예측한 결과를 어떻게 해석하면 좋을까? 사용자와 아티스트 벡터 내적이 0.5를 기준으로 그보다 높으면 모델이 선호한다고 판단했다고 말할 수 있을까?__

* 모델을 만드는 사람이 정하기 나름이다. 객관적인 지표로 만들어서 기준을 정할 수도 있고 도메인 경험을 통해 정할 수도 있다. 추천 시스템은 다른 머신러닝 task보다 객관적인 평가가 어려운 분야이다. 사람을 도통 알 수 없기 때문에 객관적인 지표를 정하기 어렵기 때문이다.

## 8. 비슷한 아티스트 찾기 + 유저에게 추천하기

### 비슷한 아티스트 찾기
---
`AlternatingLeastSquares` 클래스에 구현되어 있는 `similar_items` 메서드를 통하여 비슷한 아티스트를 찾는다. 내가 좋아하는 `coldplay`로 찾아보자.

In [25]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 0.9999999),
 (277, 0.98617357),
 (28, 0.97884214),
 (217, 0.97658926),
 (5, 0.9755894),
 (473, 0.9694854),
 (490, 0.96597),
 (247, 0.96464926),
 (418, 0.9641195),
 (694, 0.9589806),
 (55, 0.9539243),
 (910, 0.94969153),
 (1018, 0.9485097),
 (268, 0.94632566),
 (782, 0.94035435)]

(아티스트의 id, 유사도) Tuple 로 반환하고 있다. 아티스트의 id를 다시 아티스트의 이름으로 매핑 시키자.

In [26]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'radiohead',
 'red hot chili peppers',
 'placebo',
 'oasis',
 'the beatles',
 'u2',
 'foo fighters',
 'arctic monkeys',
 'nirvana',
 'the smashing pumpkins',
 'pink floyd',
 'the white stripes']

몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인해보자.

In [27]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

힙합도 좋아하기 때문에 `2pac`으로도 확인해 보자.

In [28]:
get_similar_artist('2pac')

['2pac',
 'dr. dre',
 'nas',
 'the game',
 'notorious b.i.g.',
 'jay-z',
 'ludacris',
 'ice cube',
 'snoop dogg',
 '50 cent']

오! 스눕독, 닥터 드레, 노토리우스, 나스, 제이지 같은 전설들이 추천된 모습이다. 힙합의 경우 마니아들이 많아서 더 잘 추천되는 것으로 생각된다.

__마니아는 데이터상에서 표현될 때 어떤 특징을 가질까?__

* 마니아들은 특정 장르의 아티스트들에게로 선호도가 집중되고, 다른 장르의 아티스트들과는 선호도가 낮게 나타날 것이다. 이런 마니아들의 존재로 인해 같은 장르의 아티스트들의 벡터들도 더 가까워져서 get\_similar\_artist시 장르별 특성이 두드러지게 될 것이다.

In [29]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'beyoncé',
 'the pussycat dolls',
 'kelly clarkson',
 'leona lewis',
 'christina aguilera',
 'pink']

lady gaga의 경우 여자 아티스트들이 추천되는 모습이다.

### 유저에게 아티스트 추천하기
---
`AlternatingLeastSquares` 클래스에 구현되어 있는 `recommend` 메서드를 통하여 좋아할 만한 아티스트를 추천받는다. `filter_already_liked_items` 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [30]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(350, 0.44624874),
 (550, 0.43091786),
 (369, 0.4257349),
 (274, 0.42253327),
 (354, 0.40786883),
 (1800, 0.40631527),
 (627, 0.39214587),
 (355, 0.390249),
 (2249, 0.3894663),
 (391, 0.38375628),
 (382, 0.38115662),
 (409, 0.37499446),
 (564, 0.37493175),
 (434, 0.3709092),
 (724, 0.36397406),
 (5555, 0.3620904),
 (2902, 0.36133215),
 (618, 0.35209066),
 (901, 0.35141408),
 (247, 0.3506004)]

In [31]:
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'britney spears',
 'justin timberlake',
 'michael jackson',
 'nelly furtado',
 'lady gaga',
 'maroon 5',
 'madonna',
 'katy perry',
 'christina aguilera',
 'mika',
 'amy winehouse',
 'kanye west',
 'eminem',
 'lily allen',
 'timbaland',
 'james blunt',
 'the pussycat dolls',
 'pink',
 'the beatles']

`rihanna`를 추천해주고 있다. 모델은 왜 `rihanna`를 추천해줬을까? `AlternatingLeastSquares` 클래스에 구현된 `explain` 메소드를 사용하면 제가 기록을 남긴 데이터 중 __이 추천에 기여한 정도__를 확인할 수 있다.

In [32]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수)를 반환한다. 어떤 아티스트들이 이 추천에 얼마나 기여하고 있는 걸까?

In [33]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.22630506341168866),
 ('black eyed peas', 0.1273859669689964),
 ('jason mraz', 0.04854633804728907),
 ('coldplay', 0.045579829630378696),
 ('maroon5', -0.00032869621966422893)]

`beyoncé`와 `black eyed peas`가 가장 크게 기여한 모습이다. 둘 다 흑인 아티스트이고 `beyoncé`의 경우는 같은 성별이라는 점도 크게 작용하지 않았을까 싶다.

그런데 결과가 좀 이상하다. `filter_already_liked_items=True`로 했는데 앞에 `maroon 5`는 이미 평가한 아이템이 나오고 있다. 뭘까?<br>
implicit 버전 0.4.2에서 생긴 버그이다.<br>
오픈 소스 생태계에서는 누구든 이런 [문제점을 지적](https://github.com/benfred/implicit/issues/365)하고 [Pull Request](https://github.com/benfred/implicit/pull/389)를 통하여 수정을 제안할 수 있다.
많이들 사용하는 Tensorflow나 numpy, pandas도 여러 작은 버그들이 있다. 이런 유명한 패키지의 오류를 고쳐서 Pull Request를 하여 Contribute 한다면 좋은 이력이 되기도 한다.

모델이 추천한 20개 중 몇 개를 실제로 선호하는지 계산하여 모델의 객관적인 지표를 만들 수 있을 것 같다. 자세한 방법은 [이쪽](https://danthetech.netlify.app/DataScience/evaluation-metrics-for-recommendation-system) 참조

### 마무리
---
추천시스템에서 Baseline으로 많이 사용되는 MF를 통해 유저에게 아티스트를 추천해보았다.

그러나 우리가 만든 모델은 몇 가지 아쉬운 점이 있다.

> 1. 유저, 아티스트에 대한 Meta 정보를 반영하기 쉽지 않다. 연령대별로 음악 취향이 굉장히 다르것이다.
> 2. 유저가 언제 play 했는지에 반영하기 쉽지 않다. 10년 전에 즐겨듣던 아티스트와 지금 즐겨듣는 아티스트를 비교해보자.

이러한 이유와 딥러닝의 발전으로 MF 이외의 모델 구조도 많이 연구/ 사용되고 있다. 하지만 어떤 추천 모델도 핵심은 MF와 비슷하다.<br>
유저와 아이템에 대한 벡터를 잘 학습하여 취향에 맞게(유저에 맞게) 아이템을 보여주거나(Retrieval) 걸러내는(Filtering) 역할이다.